In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
from util import *

In [2]:
data = pd.read_csv('../../input/data.csv')

In [3]:
# 数据集3：tax_info.csv
# 包含数据集7和8中涉及到的企业的纳税信息，每一行代表一个企业的纳税信息，每一行有9列，其中id列为企业唯一标识，列之间采用“,”分隔符分割。
# 数据格式如下：
# [id:企业唯一标识, START_DATE:起始时间, END_DATE:终止时间, TAX_CATEGORIES:税种, TAX_ITEMS:税目, TAXATION_BASIS:计税依据, TAX_RATE:税率, DEDUCTION:扣除数, TAX_AMOUNT:税额]
tax_info = pd.read_csv('../../input/train/tax_info.csv')
print(tax_info.shape)
tax_info.info()

(29195, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29195 entries, 0 to 29194
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              29195 non-null  object 
 1   START_DATE      29195 non-null  object 
 2   END_DATE        29195 non-null  object 
 3   TAX_CATEGORIES  29195 non-null  object 
 4   TAX_ITEMS       29195 non-null  object 
 5   TAXATION_BASIS  25816 non-null  float64
 6   TAX_RATE        25816 non-null  float64
 7   DEDUCTION       24235 non-null  float64
 8   TAX_AMOUNT      29195 non-null  float64
dtypes: float64(4), object(5)
memory usage: 2.0+ MB


In [4]:
tax_info.head(20)

,id,START_DATE,END_DATE,TAX_CATEGORIES,TAX_ITEMS,TAXATION_BASIS,TAX_RATE,DEDUCTION,TAX_AMOUNT
0,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,印花税,工伤保险（单位）,72530.75,0.0003,-0.04,21.80
1,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,印花税,失业保险（单位）,72530.75,0.0003,-0.04,21.80
2,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,印花税,医疗保险（单位）,72530.75,0.0003,-0.04,21.80
3,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,印花税,企业养老保险基金（单位）,72530.75,0.0003,-0.04,21.80
4,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,印花税,烟叶收购,72530.75,0.0003,-0.04,21.80
5,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,印花税,原油,72530.75,0.0003,-0.04,21.80
6,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,印花税,技术合同,72530.75,0.0003,-0.04,21.80
7,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,城市维护建设税,金融保险营业税额,2112.56,0.0700,0.00,147.88
8,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,城市维护建设税,优惠税率１,2112.56,0.0700,0.00,147.88
9,f000950527a6feb6c2f40c9d8477e73a439dfa0897830397,2015/09/01,2015/09/30,城市维护建设税,铁路运输,2112.56,0.0700,0.00,147.88


In [5]:
tax_info['TAX_CATEGORIES'].nunique(), tax_info['TAX_ITEMS'].nunique(), tax_info['TAX_RATE'].nunique(), tax_info['DEDUCTION'].nunique()

(17, 275, 28, 247)

In [6]:
tax_info['TAX_CATEGORIES'].unique(), tax_info['TAX_ITEMS'].unique(), tax_info['DEDUCTION'].unique()

(array(['印花税', '城市维护建设税', '教育费附加', '地方教育附加', '水利建设专项收入', '个人所得税', '企业所得税',
        '营业税', '房产税', '城镇土地使用税', '契税', '土地增值税', '残疾人就业保障金', '税务部门罚没收入',
        '耕地占用税', '其他收入', '其他专项收入'], dtype=object),
 array(['工伤保险（单位）', '失业保险（单位）', '医疗保险（单位）', '企业养老保险基金（单位）', '烟叶收购', '原油',
        '技术合同', '金融保险营业税额', '优惠税率１', '铁路运输', '一类地区', '增值税附征(市区)',
        '土地增值额（未超过50%）', '20人以上汽车及2.2升以上小轿车', '商业销售收入', '偶然所得', '住宿费',
        '大额医疗统筹(单位)', '职工标准工资', '计费折旧总值', '营业税随征', '残疾人保障金', '税务代保管资金利息',
        '城镇居民医疗保险费', '生育保险（机关事业）', '增值税教育费附加', '增值税地方教育附加', '市区（增值税附征）',
        '地方水利建设基金', '工资薪金所得', '滞纳金', '资金账簿', '购销合同', '营业税附征', '增值税随征',
        '市区、县、镇 1%', '土地增值税', '一等', '机关事业单位养老保险基金（全额单位）', '代收工会经费（2％）',
        '生猪', '各类非银行金融机构营业收入', '营业额', '生产用房', '餐饮、住宿、娱乐营业额随征', '增值税附征',
        '国有土地使用权出让', '发票违章罚款', '娱乐业', '工伤保险趸交', '应纳税所得额（随征）',
        '城镇居民医疗保险费利息', '生育保险利息', '失业保险利息', '医疗保险利息', '养老保险利息', '其他',
        '建筑业营业收入(个)', '建筑', '土地使用权转让（出售）', '一类地区（占用基本农田）', '增值税附征(县城、镇)',
        '各类非银行金融机构营业收入(全

In [7]:
def identify_missing(df, missing_threshold):
    """
    缺失率
    @param df:
    @param missing_threshold:
    @return:
    """
    missing_rate = df.isnull().sum() / len(df)
    missing_rate = missing_rate.sort_values(ascending=False)
    print(missing_rate)
    to_drop = missing_rate[missing_rate > missing_threshold].index.to_list()
    print('{} features with greater than {} missing values.\n'.format(len(to_drop), missing_threshold))
    return to_drop

In [8]:
to_drop = identify_missing(tax_info, missing_threshold=0.5)
tax_info.drop(to_drop, axis=1, inplace=True)
to_drop

DEDUCTION         0.169892
TAX_RATE          0.115739
TAXATION_BASIS    0.115739
TAX_AMOUNT        0.000000
TAX_ITEMS         0.000000
TAX_CATEGORIES    0.000000
END_DATE          0.000000
START_DATE        0.000000
id                0.000000
dtype: float64
0 features with greater than 0.5 missing values.



[]

In [9]:
tax_info['id'].nunique()

808

In [10]:
df = pd.DataFrame(tax_info['id'].unique())
df.columns = ['id']
df.shape

(808, 1)

In [11]:
tmp = tax_info.groupby('id', as_index=False)['TAXATION_BASIS'].agg({
    'TAXATION_BASIS_sum': 'sum'
})

df = df.merge(tmp, on='id', how='left')

In [12]:
tmp = tax_info.groupby('id', as_index=False)['TAX_AMOUNT'].agg({
    'TAX_AMOUNT_sum': 'sum'
})

df = df.merge(tmp, on='id', how='left')

In [13]:
df['TAX_AMOUNT_sum/TAXATION_BASIS_sum'] = df['TAX_AMOUNT_sum'] / df['TAXATION_BASIS_sum']

In [14]:
data = data.merge(df, how='left', on='id')

In [15]:
train = data[data['label'].notnull()]
test = data[data['label'].isnull()]
train.shape, test.shape

((14865, 5), (10000, 5))

In [16]:
used_cols = [i for i in train.columns if i not in ['id', 'label']]
y = train['label']
train = train[used_cols]
test = test[used_cols]

In [17]:
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.25, random_state=2020)

cols = X_train.columns
useful_dict, useless_dict, useful_cols, useless_cols = auc_select(X_train, y_train, X_valid, y_valid, cols, threshold=0.52)
print('AUC useless_cols: \n', useless_cols)

TAXATION_BASIS_sum
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's auc: 0.532403	valid_0's auc: 0.541483
Evaluated only: auc
**********
0.5414827734325216
TAX_AMOUNT_sum
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[3]	training's auc: 0.532674	valid_0's auc: 0.541695
Evaluated only: auc
**********
0.541694594836986
TAX_AMOUNT_sum/TAXATION_BASIS_sum
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's auc: 0.532595	valid_0's auc: 0.541771
Evaluated only: auc
**********
0.541770708934969
AUC useless_cols: 
 []


In [18]:
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid, reference=dtrain)

params = {
    'objective': 'binary',
    'boosting': 'gbdt',
    'metric': 'auc',
#     'metric': 'None',  # 用自定义评估函数是将metric设置为'None'
    'learning_rate': 0.1,
    'num_leaves': 31,
    'lambda_l1': 0,
    'lambda_l2': 1,
    'num_threads': 23,
    'min_data_in_leaf': 20,
    'first_metric_only': True,
    'is_unbalance': True,
    'max_depth': -1,
    'seed': 2020
}

valid_model = lgb.train(
    params,
    dtrain,
    valid_sets=[dtrain, dvalid],
    early_stopping_rounds=50,
    verbose_eval=300 
)

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	training's auc: 0.532595	valid_1's auc: 0.541627
Evaluated only: auc


In [19]:
importance = valid_model.feature_importance(importance_type='gain')
feature_name = valid_model.feature_name()

df_importance = pd.DataFrame({
    'feature_name': feature_name,
    'importance': importance
}).sort_values(by='importance', ascending=False)
df_importance['normalized_importance'] = df_importance['importance'] / df_importance['importance'].sum()
df_importance['cumulative_importance'] = np.cumsum(df_importance['normalized_importance'])
df_importance

,feature_name,importance,normalized_importance,cumulative_importance
1,TAX_AMOUNT_sum,2190.090088,1.0,1.0
0,TAXATION_BASIS_sum,0.000000,0.0,1.0
2,TAX_AMOUNT_sum/TAXATION_BASIS_sum,0.000000,0.0,1.0
